# Creacion Consolidado por Materia Penal

Comenzaremos el analisis de la data consolidando la info en un unico DataSet, para ello vamos a considerar los siguientes puntos:


- DATA:  Se considerará toda las causas que ingresaron al Sistema desde 2015 al 2019, esto ya que se cuenta además con la data de las audiencias en ese período.


- No se consideran causas anteriores a este período.

- Se considerarán Causas sin termino.

In [1]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from unicodedata import normalize
from src.data import cleandata
from src.data import transformdata

In [2]:
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
path_processed = "../data/processed/pjud"

#### Paso 1 : Consolidacion de Dataset x Materia y x rol

In [5]:
df_ingresos_materia = pd.read_feather(f"{path_processed}/IngresosMateria_feather")
print(f"{len(df_ingresos_materia)} causas en el dataset de ingresos por materia")

3210643 causas en el dataset de ingresos por materia


In [6]:
df_termino_materia = pd.read_feather(f"{path_processed}/TerminoMateria_feather")
print(f"Existen : {len(df_termino_materia)} causas en el dataset de termino por materia")

Existen : 3143099 causas en el dataset de termino por materia


Analizamos la data, uniendo ambos dataset.

In [7]:
df_fulldata_materia = pd.merge(df_ingresos_materia, df_termino_materia, how='outer', on=['COD. TRIBUNAL','RIT','COD. MATERIA'])
print(f"Existen: {len(df_fulldata_materia)} causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso")

Existen: 4249511 causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso


In [8]:
df_causas_terminadas = pd.merge(df_ingresos_materia, df_termino_materia, how='inner', on=['COD. TRIBUNAL','RIT','COD. MATERIA'])
print(f"Existen: {len(df_causas_terminadas)} causas ingresadas y terminadas")

Existen: 2318621 causas ingresadas y terminadas


### Analisis de la Data Causas por Materia

De estos dos dataset debo considerar:

- Caso 1: el dataset df_causas_terminadas completo
- Caso 2: Debo agregar además las causas que tengan Fecha Ingreso_x pero que aún no terminan.
- Caso 3: Debo analizar las causas que no tienen Fecha Ingreso_x pero si tienen Fecha Ingreso_y y Fecha Ingreso_y, ya que deben ser las causas que no fueron tomadas por la consulta de Ingresos. Acá se debe reemplazar los datos obtenidos en Termino Materia a Ingresos.

Luego de tener estos tres dataset debo concatenarlos

-----------------------------------------------------------------

### Caso 1:

In [9]:
df_causas_terminadas.columns

Index(['level_0_x', 'index_x', 'COD. CORTE_x', 'CORTE_x', 'COD. TRIBUNAL',
       'TRIBUNAL_x', 'RIT', 'TIPO CAUSA_x', 'COD. MATERIA', 'MATERIA_x',
       'FECHA INGRESO_x', 'MES INGRESO', 'AÑO INGRESO',
       'TOTAL INGRESOS POR MATERIAS', 'level_0_y', 'index_y', 'COD. CORTE_y',
       'CORTE_y', 'TRIBUNAL_y', 'TIPO CAUSA_y', 'MATERIA_y', 'FECHA INGRESO_y',
       'FECHA TERMINO', 'MES TERMINO', 'AÑO TERMINO', 'MOTIVO TERMINO',
       'DURACION CAUSA', 'TOTAL TERMINOS'],
      dtype='object')

Eliminare columnas duplicadas para formar el dataset consolidado.

In [10]:
columnas_drop = ['level_0_x', 'index_x', 'COD. CORTE_x', 'CORTE_x', 'TRIBUNAL_x', 'TIPO CAUSA_x','level_0_y', 'index_y']

df_causas_terminadas.drop(columnas_drop, axis = 'columns', inplace = True)

In [11]:
df_causas_terminadas.rename(columns = {'COD. CORTE_y':'COD. CORTE',
                                        'CORTE_y':'CORTE',
                                        'TRIBUNAL_y':'TRIBUNAL',
                                        'TIPO CAUSA_y':'TIPO CAUSA',
                                        'MATERIA_x':'MATERIA IM',
                                        'FECHA INGRESO_x':'FECHA INGRESO IM',
                                        'FECHA INGRESO_y':'FECHA INGRESO TM',
                                        'MATERIA_y':'MATERIA TM',
                                       }, inplace = True)

In [12]:
df_causas_terminadas['MOTIVO TERMINO'].unique()

array(['SENTENCIA', 'DECLARA SOBRESEIMIENTO DEFINITIVO.',
       'COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNIDAD.',
       'DECLARA INADMISIBILIDAD DE LA QUERELLA.',
       'APROBACION NO INICIO INVESTIGACION.',
       'ACOGE REQUERIMIENTO (MONITORIO).',
       'NO PERSEVERAR EN EL PROCEDIMIENTO.',
       'CERTIFICA CUMPLIMIENTO ART. 468.', 'ABANDONO DE LA QUERELLA',
       'ACUMULACION.', 'DECLARA INCOMPETENCIA.',
       'DECLARA INCOMPETENCIA RPA', 'DECLARA SOBRESEIMIENTO DEFINITIVO',
       'COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNIDAD'], dtype=object)

In [13]:
df_causas_terminadas['MOTIVO TERMINO'].value_counts()

APROBACION NO INICIO INVESTIGACION.                   508135
SENTENCIA                                             443854
DECLARA SOBRESEIMIENTO DEFINITIVO.                    405479
ACOGE REQUERIMIENTO (MONITORIO).                      253851
COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNIDAD.    215284
DECLARA SOBRESEIMIENTO DEFINITIVO                     149184
NO PERSEVERAR EN EL PROCEDIMIENTO.                    144665
COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNIDAD      67731
CERTIFICA CUMPLIMIENTO ART. 468.                       41101
ACUMULACION.                                           37121
DECLARA INCOMPETENCIA.                                 28345
DECLARA INCOMPETENCIA RPA                              11991
DECLARA INADMISIBILIDAD DE LA QUERELLA.                 7933
ABANDONO DE LA QUERELLA                                 3947
Name: MOTIVO TERMINO, dtype: int64

Aplico transformación de motivo termino, ya que existen duplicidad de nombres.

In [14]:
df_causas_terminadas['MOTIVO TERMINO'] = df_causas_terminadas['MOTIVO TERMINO'].progress_apply(cleandata.cambio_termino_causa)

100%|██████████| 2318621/2318621 [00:02<00:00, 1107804.24it/s]


Acá analizamos si existe alguna relación entre Total Ingresos por Matería y Total Terminos.

In [15]:
df_causas_terminadas[df_causas_terminadas['TOTAL INGRESOS POR MATERIAS']!=df_causas_terminadas['TOTAL TERMINOS']]

,COD. TRIBUNAL,RIT,COD. MATERIA,MATERIA IM,FECHA INGRESO IM,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,COD. CORTE,CORTE,TRIBUNAL,TIPO CAUSA,MATERIA TM,FECHA INGRESO TM,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
29,988,120-2018,702,HOMICIDIO.,2018-04-16,abr-18,2018,2,10,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,HOMICIDIO.,2018-04-16,2018-06-30,jun-18,2018,SENTENCIA,75,1
39,988,128-2018,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-19,abr-18,2018,2,10,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-19,2018-06-11,jun-18,2018,SENTENCIA,53,1
59,988,145-2018,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-04-24,abr-18,2018,3,10,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-04-24,2018-06-26,jun-18,2018,SENTENCIA,63,1
62,988,149-2018,524,AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES...,2018-04-26,abr-18,2018,2,10,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES...,2018-04-26,2018-07-02,jul-18,2018,SENTENCIA,67,1
79,988,158-2018,812,RECEPTACION. ART. 456 BIS A.,2018-05-02,may-18,2018,2,10,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,RECEPTACION. ART. 456 BIS A.,2018-05-02,2018-05-30,may-18,2018,SENTENCIA,28,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318570,1355,68-2015,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2015-09-24,sept-15,2015,3,91,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2015-09-24,2016-01-05,ene-16,2016,SENTENCIA,103,1
2318584,1355,80-2015,802,ROBO CON INTIMIDACION.,2015-10-28,oct-15,2015,3,91,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,ROBO CON INTIMIDACION,2015-10-28,2015-11-20,nov-15,2015,SENTENCIA,23,1
2318585,1355,81-2015,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2015-10-28,oct-15,2015,2,91,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIVIL),2015-10-28,2015-12-18,dic-15,2015,SENTENCIA,51,1
2318605,1355,92-2015,14060,OTROS DELITOS CONTRA LA LEY DEL TRANSITO.,2015-12-04,dic-15,2015,2,91,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,OTROS DELITOS CONTRA LA LEY DEL TRANSITO.,2015-12-04,2016-05-27,may-16,2016,SENTENCIA,175,1


Vemos que existen cerca de 30 mil ingresos por materia donde se diferencian... ¿cual es el motivo?

- Se puede deber a que pueden ser más de un imputado, donde se termino para uno de ellos.

-----------------------------------------------------------------

### Caso 2:

In [16]:
filtro_sintermino = df_fulldata_materia['FECHA INGRESO_x'].notnull() & df_fulldata_materia['FECHA TERMINO'].isnull()
df_causas_sintermino = df_fulldata_materia.loc[filtro_sintermino]

In [17]:
df_causas_sintermino.columns

Index(['level_0_x', 'index_x', 'COD. CORTE_x', 'CORTE_x', 'COD. TRIBUNAL',
       'TRIBUNAL_x', 'RIT', 'TIPO CAUSA_x', 'COD. MATERIA', 'MATERIA_x',
       'FECHA INGRESO_x', 'MES INGRESO', 'AÑO INGRESO',
       'TOTAL INGRESOS POR MATERIAS', 'level_0_y', 'index_y', 'COD. CORTE_y',
       'CORTE_y', 'TRIBUNAL_y', 'TIPO CAUSA_y', 'MATERIA_y', 'FECHA INGRESO_y',
       'FECHA TERMINO', 'MES TERMINO', 'AÑO TERMINO', 'MOTIVO TERMINO',
       'DURACION CAUSA', 'TOTAL TERMINOS'],
      dtype='object')

In [18]:
columnas_drop = ['level_0_x', 'index_x','level_0_y', 'index_y', 'COD. CORTE_y', 'CORTE_y','TRIBUNAL_y', 'TIPO CAUSA_y']

df_causas_sintermino.drop(columnas_drop, axis='columns', inplace=True)

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
df_causas_sintermino.rename(columns = {'COD. CORTE_x':'COD. CORTE',
                                        'CORTE_x':'CORTE',
                                        'TRIBUNAL_x':'TRIBUNAL',
                                        'TIPO CAUSA_x':'TIPO CAUSA',
                                        'MATERIA_x':'MATERIA IM',
                                        'MATERIA_y':'MATERIA TM',
                                        'FECHA INGRESO_x':'FECHA INGRESO IM',
                                        'FECHA INGRESO_y':'FECHA INGRESO TM'
                                       }, inplace = True)

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
df_causas_sintermino

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA IM,FECHA INGRESO IM,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,MATERIA TM,FECHA INGRESO TM,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
21,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,113-2018,Ordinaria,14004,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2018-04-11,abr-18,2018.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
47,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,134-2018,Ordinaria,7038,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,2018-04-19,abr-18,2018.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
48,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,134-2018,Ordinaria,14004,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2018-04-19,abr-18,2018.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
49,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,135-2018,Ordinaria,12072,FALSIFICACION LICENCIA DE CONDUCIR Y OTRAS FAL...,2018-04-19,abr-18,2018.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
65,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,147-2018,Ordinaria,623,ABUSO SEXUAL CON CONTACTO DE MENOR DE 14 AÑOS....,2018-04-26,abr-18,2018.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423156,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,21-2015,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2015-03-11,mar-15,2015.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3423224,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,67-2015,Ordinaria,828,ROBO CON VIOLACION.,2015-09-22,sept-15,2015.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3423225,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,67-2015,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-09-22,sept-15,2015.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3423238,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,75-2015,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2015-10-13,oct-15,2015.0,1.0,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN


Aca podemos completar algunos datos faltantes utilizando los datos del dataset INGRESOS:

    - FECHA INGRESO IM en Termino
    - MATERIA

In [21]:
df_causas_sintermino = df_causas_sintermino.progress_apply(transformdata.faltantes_materia, axis=1)

100%|██████████| 1104663/1104663 [03:56<00:00, 4674.03it/s]


In [22]:
df_causas_sintermino

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA IM,FECHA INGRESO IM,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,MATERIA TM,FECHA INGRESO TM,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
21,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,113-2018,Ordinaria,14004,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2018-04-11,abr-18,2018.0,1.0,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2018-04-11,NaT,NaN,NaN,NaN,NaN,NaN
47,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,134-2018,Ordinaria,7038,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,2018-04-19,abr-18,2018.0,1.0,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,2018-04-19,NaT,NaN,NaN,NaN,NaN,NaN
48,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,134-2018,Ordinaria,14004,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2018-04-19,abr-18,2018.0,1.0,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2018-04-19,NaT,NaN,NaN,NaN,NaN,NaN
49,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,135-2018,Ordinaria,12072,FALSIFICACION LICENCIA DE CONDUCIR Y OTRAS FAL...,2018-04-19,abr-18,2018.0,1.0,FALSIFICACION LICENCIA DE CONDUCIR Y OTRAS FAL...,2018-04-19,NaT,NaN,NaN,NaN,NaN,NaN
65,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,147-2018,Ordinaria,623,ABUSO SEXUAL CON CONTACTO DE MENOR DE 14 AÑOS....,2018-04-26,abr-18,2018.0,1.0,ABUSO SEXUAL CON CONTACTO DE MENOR DE 14 AÑOS....,2018-04-26,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3423156,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,21-2015,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2015-03-11,mar-15,2015.0,1.0,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2015-03-11,NaT,NaN,NaN,NaN,NaN,NaN
3423224,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,67-2015,Ordinaria,828,ROBO CON VIOLACION.,2015-09-22,sept-15,2015.0,1.0,ROBO CON VIOLACION.,2015-09-22,NaT,NaN,NaN,NaN,NaN,NaN
3423225,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,67-2015,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-09-22,sept-15,2015.0,1.0,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-09-22,NaT,NaN,NaN,NaN,NaN,NaN
3423238,91.0,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,75-2015,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2015-10-13,oct-15,2015.0,1.0,TRAFICO ILICITO DE DROGAS (ART. 3).,2015-10-13,NaT,NaN,NaN,NaN,NaN,NaN


-----------------------------------------------------------------

### Caso 3:

In [23]:
filtro_sininicio = df_fulldata_materia['FECHA INGRESO_x'].isnull() & df_fulldata_materia['FECHA TERMINO'].notnull() 
df_causas_sininicio = df_fulldata_materia.loc[filtro_sininicio]
df_causas_sininicio.columns

Index(['level_0_x', 'index_x', 'COD. CORTE_x', 'CORTE_x', 'COD. TRIBUNAL',
       'TRIBUNAL_x', 'RIT', 'TIPO CAUSA_x', 'COD. MATERIA', 'MATERIA_x',
       'FECHA INGRESO_x', 'MES INGRESO', 'AÑO INGRESO',
       'TOTAL INGRESOS POR MATERIAS', 'level_0_y', 'index_y', 'COD. CORTE_y',
       'CORTE_y', 'TRIBUNAL_y', 'TIPO CAUSA_y', 'MATERIA_y', 'FECHA INGRESO_y',
       'FECHA TERMINO', 'MES TERMINO', 'AÑO TERMINO', 'MOTIVO TERMINO',
       'DURACION CAUSA', 'TOTAL TERMINOS'],
      dtype='object')

In [24]:
columnas_drop = ['level_0_x', 'index_x', 'COD. CORTE_x', 'CORTE_x','TRIBUNAL_x', 'TIPO CAUSA_x', 'level_0_y', 'index_y']
df_causas_sininicio.drop(columnas_drop, axis='columns', inplace=True)

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
df_causas_sininicio.rename(columns = {'COD. CORTE_y':'COD. CORTE',
                                      'CORTE_y':'CORTE',
                                      'TRIBUNAL_y':'TRIBUNAL',
                                      'TIPO CAUSA_y':'TIPO CAUSA',
                                      'MATERIA_y':'MATERIA TM',
                                      'MATERIA_x':'MATERIA IM',
                                      'FECHA INGRESO_y':'FECHA INGRESO TM',
                                      'FECHA INGRESO_x':'FECHA INGRESO IM'
                                       }, inplace = True)

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
df_causas_sininicio

,COD. TRIBUNAL,RIT,COD. MATERIA,MATERIA IM,FECHA INGRESO IM,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,COD. CORTE,CORTE,TRIBUNAL,TIPO CAUSA,MATERIA TM,FECHA INGRESO TM,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
3423284,988,233-2017,13036,NaN,NaT,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,LESIONES LEVES 494 N°5 CODIGO PENAL,2017-07-12,2018-01-10,ene-18,2018.0,SENTENCIA,182.0,1.0
3423285,988,329-2016,14004,NaN,NaT,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2016-10-20,2018-08-14,ago-18,2018.0,SENTENCIA,663.0,1.0
3423286,988,388-2017,7038,NaN,NaT,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,2017-10-18,2018-01-30,ene-18,2018.0,SENTENCIA,104.0,1.0
3423287,988,404-2017,419,NaN,NaT,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,FRAUDES AL FISCO Y ORGANISMOS DEL ESTADO (ART....,2017-10-25,2018-12-21,dic-18,2018.0,SENTENCIA,422.0,1.0
3423288,988,424-2017,847,NaN,NaT,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,HURTO SIMPLE POR UN VALOR DE 4 A 40 UTM.,2017-11-10,2018-01-09,ene-18,2018.0,SENTENCIA,60.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249506,1323,247-2015,303,NaN,NaT,NaN,NaN,NaN,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,FALSIFICACION O USO MALICIOSO DE DOCUMENTOS PR...,2015-12-09,2016-03-24,mar-16,2016.0,SENTENCIA,106.0,1.0
4249507,1323,251-2015,7037,NaN,NaT,NaN,NaN,NaN,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0
4249508,1323,251-2015,10009,NaN,NaT,NaN,NaN,NaN,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,POSESION O TENENCIA DE ARMAS PROHIBIDAS,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0
4249509,1323,251-2015,10099,NaN,NaT,NaN,NaN,NaN,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0


In [27]:
df_causas_sininicio['MOTIVO TERMINO'].value_counts()

DECLARA SOBRESEIMIENTO DEFINITIVO.                    198818
SENTENCIA                                             149748
APROBACION NO INICIO INVESTIGACION.                   131684
COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNIDAD.     84226
DECLARA INCOMPETENCIA.                                 71643
NO PERSEVERAR EN EL PROCEDIMIENTO.                     67066
ACOGE REQUERIMIENTO (MONITORIO).                       64186
CERTIFICA CUMPLIMIENTO ART. 468.                       23909
ACUMULACION.                                           16893
DECLARA INCOMPETENCIA RPA                              11480
ABANDONO DE LA QUERELLA                                 2025
DECLARA SOBRESEIMIENTO DEFINITIVO                       1994
DECLARA INADMISIBILIDAD DE LA QUERELLA.                 1973
COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNIDAD        582
Name: MOTIVO TERMINO, dtype: int64

In [28]:
df_causas_sininicio['MOTIVO TERMINO'] = df_causas_sininicio['MOTIVO TERMINO'].progress_apply(cleandata.cambio_termino_causa)

100%|██████████| 826227/826227 [00:00<00:00, 1068340.69it/s]
/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Aca podemos completar algunos datos faltantes utilizando los datos del dataset TERMINOS:

    - FECHA INGRESO TM en INGRESO
    - MATERIA
    

In [31]:
df_causas_sininicio = df_causas_sininicio.progress_apply(transformdata.faltantes_materia, axis=1)

100%|██████████| 826227/826227 [02:56<00:00, 4671.51it/s]


In [32]:
df_causas_sininicio

,COD. TRIBUNAL,RIT,COD. MATERIA,MATERIA IM,FECHA INGRESO IM,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,COD. CORTE,CORTE,TRIBUNAL,TIPO CAUSA,MATERIA TM,FECHA INGRESO TM,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
3423284,988,233-2017,13036,LESIONES LEVES 494 N°5 CODIGO PENAL,2017-07-12,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,LESIONES LEVES 494 N°5 CODIGO PENAL,2017-07-12,2018-01-10,ene-18,2018.0,SENTENCIA,182.0,1.0
3423285,988,329-2016,14004,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2016-10-20,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2016-10-20,2018-08-14,ago-18,2018.0,SENTENCIA,663.0,1.0
3423286,988,388-2017,7038,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,2017-10-18,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,2017-10-18,2018-01-30,ene-18,2018.0,SENTENCIA,104.0,1.0
3423287,988,404-2017,419,FRAUDES AL FISCO Y ORGANISMOS DEL ESTADO (ART....,2017-10-25,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,FRAUDES AL FISCO Y ORGANISMOS DEL ESTADO (ART....,2017-10-25,2018-12-21,dic-18,2018.0,SENTENCIA,422.0,1.0
3423288,988,424-2017,847,HURTO SIMPLE POR UN VALOR DE 4 A 40 UTM.,2017-11-10,NaN,NaN,NaN,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,HURTO SIMPLE POR UN VALOR DE 4 A 40 UTM.,2017-11-10,2018-01-09,ene-18,2018.0,SENTENCIA,60.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249506,1323,247-2015,303,FALSIFICACION O USO MALICIOSO DE DOCUMENTOS PR...,2015-12-09,NaN,NaN,NaN,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,FALSIFICACION O USO MALICIOSO DE DOCUMENTOS PR...,2015-12-09,2016-03-24,mar-16,2016.0,SENTENCIA,106.0,1.0
4249507,1323,251-2015,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-12-10,NaN,NaN,NaN,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0
4249508,1323,251-2015,10009,POSESION O TENENCIA DE ARMAS PROHIBIDAS,2015-12-10,NaN,NaN,NaN,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,POSESION O TENENCIA DE ARMAS PROHIBIDAS,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0
4249509,1323,251-2015,10099,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,2015-12-10,NaN,NaN,NaN,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0


-----------------------------------------------------------------

### Concatenar los dataset relacionados a MATERIA

In [36]:
df_consolidado_materia = pd.concat([df_causas_terminadas,df_causas_sininicio,df_causas_sintermino], axis=0, ignore_index=True)

In [37]:
df_consolidado_materia

,COD. TRIBUNAL,RIT,COD. MATERIA,MATERIA IM,FECHA INGRESO IM,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,COD. CORTE,CORTE,TRIBUNAL,TIPO CAUSA,MATERIA TM,FECHA INGRESO TM,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
0,988,100-2018,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,abr-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018-06-01,jun-18,2018.0,SENTENCIA,60.0,1.0
1,988,100-2018,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,abr-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018-06-01,jun-18,2018.0,SENTENCIA,60.0,1.0
2,988,101-2018,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,abr-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018-08-29,ago-18,2018.0,SENTENCIA,149.0,1.0
3,988,10-2018,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,ene-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018-03-14,mar-18,2018.0,SENTENCIA,63.0,1.0
4,988,102-2018,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,abr-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018-05-29,may-18,2018.0,SENTENCIA,57.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249506,1355,21-2015,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2015-03-11,mar-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2015-03-11,NaT,NaN,NaN,NaN,NaN,NaN
4249507,1355,67-2015,828,ROBO CON VIOLACION.,2015-09-22,sept-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,ROBO CON VIOLACION.,2015-09-22,NaT,NaN,NaN,NaN,NaN,NaN
4249508,1355,67-2015,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-09-22,sept-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-09-22,NaT,NaN,NaN,NaN,NaN,NaN
4249509,1355,75-2015,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2015-10-13,oct-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,TRAFICO ILICITO DE DROGAS (ART. 3).,2015-10-13,NaT,NaN,NaN,NaN,NaN,NaN


Proceso a transformar nombres de MOTIVO TERMINO

In [38]:
df_consolidado_materia.dtypes

COD. TRIBUNAL                           int64
RIT                                    object
COD. MATERIA                            int64
MATERIA IM                             object
FECHA INGRESO IM               datetime64[ns]
MES INGRESO                            object
AÑO INGRESO                           float64
TOTAL INGRESOS POR MATERIAS           float64
COD. CORTE                            float64
CORTE                                  object
TRIBUNAL                               object
TIPO CAUSA                             object
MATERIA TM                             object
FECHA INGRESO TM               datetime64[ns]
FECHA TERMINO                  datetime64[ns]
MES TERMINO                            object
AÑO TERMINO                           float64
MOTIVO TERMINO                         object
DURACION CAUSA                        float64
TOTAL TERMINOS                        float64
dtype: object

In [39]:
df_consolidado_materia['MOTIVO TERMINO'].unique()

array(['SENTENCIA', 'DECLARA SOBRESEIMIENTO DEFINITIVO',
       'COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD',
       'DECLARA INADMISIBILIDAD DE LA QUERELLA',
       'APROBACION NO INICIO INVESTIGACION',
       'ACOGE REQUERIMIENTO (MONITORIO)',
       'NO PERSEVERAR EN EL PROCEDIMIENTO',
       'CERTIFICA CUMPLIMIENTO ART 468', 'ABANDONO DE LA QUERELLA',
       'ACUMULACION', 'DECLARA INCOMPETENCIA',
       'DECLARA INCOMPETENCIA RPA', nan], dtype=object)

In [40]:
df_consolidado_materia['MOTIVO TERMINO'].value_counts()

DECLARA SOBRESEIMIENTO DEFINITIVO                   755475
APROBACION NO INICIO INVESTIGACION                  639819
SENTENCIA                                           593602
COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD    367823
ACOGE REQUERIMIENTO (MONITORIO)                     318037
NO PERSEVERAR EN EL PROCEDIMIENTO                   211731
DECLARA INCOMPETENCIA                                99988
CERTIFICA CUMPLIMIENTO ART 468                       65010
ACUMULACION                                          54014
DECLARA INCOMPETENCIA RPA                            23471
DECLARA INADMISIBILIDAD DE LA QUERELLA                9906
ABANDONO DE LA QUERELLA                               5972
Name: MOTIVO TERMINO, dtype: int64

-----------------------------------------------------------------

In [41]:
filtro_oral = df_consolidado_materia[df_consolidado_materia['TRIBUNAL'].str.contains('ORAL')]
filtro_garantia = df_consolidado_materia[df_consolidado_materia['TRIBUNAL'].str.contains('GARANTIA')]

In [42]:
filtro_oral

,COD. TRIBUNAL,RIT,COD. MATERIA,MATERIA IM,FECHA INGRESO IM,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,COD. CORTE,CORTE,TRIBUNAL,TIPO CAUSA,MATERIA TM,FECHA INGRESO TM,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
0,988,100-2018,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,abr-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018-06-01,jun-18,2018.0,SENTENCIA,60.0,1.0
1,988,100-2018,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,abr-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018-06-01,jun-18,2018.0,SENTENCIA,60.0,1.0
2,988,101-2018,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,abr-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018-08-29,ago-18,2018.0,SENTENCIA,149.0,1.0
3,988,10-2018,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,ene-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018-03-14,mar-18,2018.0,SENTENCIA,63.0,1.0
4,988,102-2018,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,abr-18,2018.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018-05-29,may-18,2018.0,SENTENCIA,57.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249506,1355,21-2015,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2015-03-11,mar-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2015-03-11,NaT,NaN,NaN,NaN,NaN,NaN
4249507,1355,67-2015,828,ROBO CON VIOLACION.,2015-09-22,sept-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,ROBO CON VIOLACION.,2015-09-22,NaT,NaN,NaN,NaN,NaN,NaN
4249508,1355,67-2015,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-09-22,sept-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-09-22,NaT,NaN,NaN,NaN,NaN,NaN
4249509,1355,75-2015,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2015-10-13,oct-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,Ordinaria,TRAFICO ILICITO DE DROGAS (ART. 3).,2015-10-13,NaT,NaN,NaN,NaN,NaN,NaN


In [43]:
filtro_garantia

,COD. TRIBUNAL,RIT,COD. MATERIA,MATERIA IM,FECHA INGRESO IM,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,COD. CORTE,CORTE,TRIBUNAL,TIPO CAUSA,MATERIA TM,FECHA INGRESO TM,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
467,992,1000-2018,13028,HURTO FALTA (494 BIS CODIGO PENAL).,2018-02-01,feb-18,2018.0,1.0,10.0,C.A. DE ARICA,JUZGADO DE GARANTIA ARICA,Ordinaria,HURTO FALTA (494 BIS CODIGO PENAL).,2018-02-01,2018-02-08,feb-18,2018.0,COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD,7.0,1.0
468,992,10004-2018,524,AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES...,2018-12-06,dic-18,2018.0,1.0,10.0,C.A. DE ARICA,JUZGADO DE GARANTIA ARICA,Ordinaria,AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES...,2018-12-06,2018-12-13,dic-18,2018.0,DECLARA INADMISIBILIDAD DE LA QUERELLA,7.0,1.0
469,992,10004-2018,710,LESIONES MENOS GRAVES.,2018-12-06,dic-18,2018.0,1.0,10.0,C.A. DE ARICA,JUZGADO DE GARANTIA ARICA,Ordinaria,LESIONES MENOS GRAVES.,2018-12-06,2018-12-13,dic-18,2018.0,DECLARA INADMISIBILIDAD DE LA QUERELLA,7.0,1.0
470,992,1001-2018,840,DAÑOS SIMPLES.,2018-02-02,feb-18,2018.0,1.0,10.0,C.A. DE ARICA,JUZGADO DE GARANTIA ARICA,Ordinaria,DAÑOS SIMPLES.,2018-02-02,2018-02-02,feb-18,2018.0,APROBACION NO INICIO INVESTIGACION,0.0,1.0
471,992,10013-2018,13028,HURTO FALTA (494 BIS CODIGO PENAL).,2018-12-08,dic-18,2018.0,1.0,10.0,C.A. DE ARICA,JUZGADO DE GARANTIA ARICA,Ordinaria,HURTO FALTA (494 BIS CODIGO PENAL).,2018-12-08,2018-12-08,dic-18,2018.0,SENTENCIA,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249427,1240,6626-2015,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2015-12-30,dic-15,2015.0,2.0,91.0,C.A. DE SAN MIGUEL,JUZGADO DE GARANTIA TALAGANTE,Ordinaria,TRAFICO ILICITO DE DROGAS (ART. 3).,2015-12-30,NaT,NaN,NaN,NaN,NaN,NaN
4249428,1240,6630-2015,13028,HURTO FALTA (494 BIS CODIGO PENAL).,2015-12-30,dic-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,JUZGADO DE GARANTIA TALAGANTE,Ordinaria,HURTO FALTA (494 BIS CODIGO PENAL).,2015-12-30,NaT,NaN,NaN,NaN,NaN,NaN
4249429,1240,6631-2015,13028,HURTO FALTA (494 BIS CODIGO PENAL).,2015-12-30,dic-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,JUZGADO DE GARANTIA TALAGANTE,Ordinaria,HURTO FALTA (494 BIS CODIGO PENAL).,2015-12-30,NaT,NaN,NaN,NaN,NaN,NaN
4249430,1240,6638-2015,524,AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES...,2015-12-31,dic-15,2015.0,1.0,91.0,C.A. DE SAN MIGUEL,JUZGADO DE GARANTIA TALAGANTE,Ordinaria,AMENAZAS SIMPLES CONTRA PERSONAS Y PROPIEDADES...,2015-12-31,NaT,NaN,NaN,NaN,NaN,NaN


In [44]:
filtro_oral['TRIBUNAL'].value_counts()

TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE             6225
SEXTO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO      5992
SEGUNDO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO    5593
TRIBUNAL DE JUICIO ORAL EN LO PENAL RANCAGUA            5140
CUARTO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO     5130
TRIBUNAL DE JUICIO ORAL EN LO PENAL VALPARAISO          4839
TRIBUNAL DE JUICIO ORAL EN LO PENAL VIÑA DEL MAR        4570
TRIBUNAL DE JUICIO ORAL EN LO PENAL CONCEPCION          4441
SEPTIMO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO    3862
TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO        3722
TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA         3658
TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA               3655
TERCER TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO     3400
TRIBUNAL DE JUICIO ORAL EN LO PENAL TALCA               2913
TRIBUNAL DE JUICIO ORAL EN LO PENAL LA SERENA           2556
PRIMER TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO     2547
TRIBUNAL DE JUICIO ORAL 

In [45]:
filtro_oral['MATERIA IM'].value_counts()

TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).                       9655
TRAFICO ILICITO DE DROGAS (ART. 3).                            6882
RECEPTACION. ART. 456 BIS A.                                   6819
ROBO CON INTIMIDACION.                                         6395
DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIVIL).             5246
                                                               ... 
DELITOS CONTRA LA LEY DE BOSQUE NATIVO.LEY 20.283.                1
LOTERIA ILEGAL, CASAS DE JUEGO Y PRESTAM  SOBRE PRENDA            1
ENVIO EXPLOSIVOS,HOMICIDIO,LESIONES Y SECUESTRO TERRORISTA.       1
APODERAMIENTO O ATENTADO AL TRANSPORTE PUBLICO.                   1
EXPENDIO DE BEBIDAS ALCOHOLICAS A MENORES.                        1
Name: MATERIA IM, Length: 359, dtype: int64

In [46]:
filtro_oral['MOTIVO TERMINO'].value_counts()

SENTENCIA                            83230
DECLARA SOBRESEIMIENTO DEFINITIVO     2085
ACUMULACION                            783
DECLARA INCOMPETENCIA                  249
ABANDONO DE LA QUERELLA                 54
DECLARA INCOMPETENCIA RPA                3
Name: MOTIVO TERMINO, dtype: int64

## CARGA DATA A ARCHIVOS FUTHER

In [47]:
# Directorio donde se guardaran archivos feather
df_consolidado_materia.reset_index(inplace = True)
filtro_oral.reset_index(inplace = True)
filtro_garantia.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_consolidado_materia.to_feather(f'{path_processed}/Consolidado_Materia_feather')
filtro_oral.to_feather(f'{path_processed}/JuicioOralesMateria_feather')
filtro_garantia.to_feather(f'{path_processed}/CausasGarantiaMaterui_feather')